In [1]:
from pathlib import Path
fig_dir_path = Path(".") / "figs"
fig_dir_path.mkdir(exist_ok=True)

In [3]:
# time series and functional connectivity
import numpy as np
from matplotlib import pyplot as plt

npz_file_path = Path("..") / "run_files" / "ds002748" / "sub-01" / "features.npz"
assert npz_file_path.exists(), f"File not found: {npz_file_path}"
with np.load(npz_file_path) as data:
    features = data["time_series"].T
    fc_matrix = data["fc_matrix"]

data = features[:3]
time = np.arange(data.shape[1])

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 4), constrained_layout=False)

colors = ['#3FA7D6', '#F7B538', '#D7263D']

for i, ax in enumerate(axes):
    ax.plot(time, data[i], color=colors[i], linewidth=3.5)
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title('')
    ax.grid(False)
    ax.margins(0)   # removes the 5% padding on both axes

# Remove gaps between subplots and borders
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0)

plt.savefig(fig_dir_path / 'ts.png', dpi=900, bbox_inches='tight', pad_inches=0)
plt.close()

downsampled = fc_matrix[::2, ::2] 
downsampled = downsampled[123-66:, 123-66:]
plt.figure(figsize=(6, 6))
plt.imshow(downsampled, cmap='coolwarm', aspect='equal')
plt.axis('off') 
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.savefig(fig_dir_path / 'fc_heatmap.png', dpi=900, bbox_inches='tight', pad_inches=0)
plt.close()

In [4]:
# FPR of the 8 brain networks
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Define the fixed mapping of colors for each brain region
color_map = {
    "Visual": plt.cm.viridis(0.2),   # Assign color using Viridis colormap
    "Limbic": plt.cm.viridis(0.4),
    "Ventral": plt.cm.viridis(0.6),
    "Control": plt.cm.viridis(0.8),
    "Somato": plt.cm.viridis(0.1),
    "Dorsal": plt.cm.viridis(0.3),
    "Salience": plt.cm.viridis(0.5),
    "Default": plt.cm.viridis(1.0)
}

# Data preparation for each plot
data_sets = [
    ({"Visual": 0.1875, "Limbic": 0.2, "Ventral": 0.25, "Control": 0.25, "Somato": 0.275,
      "Dorsal": 0.275, "Salience": 0.2875, "Default": 0.3125}, 'Node_FPR.pdf', (0.18, 0.32)),
    
    ({"Visual": 0.1875, "Limbic": 0.225, "Dorsal": 0.225, "Somato": 0.2375, "Ventral": 0.2625,
      "Salience": 0.2625, "Control": 0.2875, "Default": 0.3}, 'Edge_FPR.pdf', (0.18, 0.32))
]

# Function to create each plot
def create_plot(data, filename, ylim):
    categories = list(data.keys())
    values = list(data.values())
    colors = [color_map[category] for category in categories]  # Use the fixed color mapping

    plt.figure(figsize=(10, 6))
    plt.bar(categories, values, color=colors)
    plt.ylim(*ylim)  # Set y-axis limits
    plt.yticks(np.arange(ylim[0], ylim[1], 0.02), fontsize=16, fontweight='bold')
    plt.xticks(rotation=0, fontname='Times New Roman', fontsize=18, fontweight='bold')
    plt.grid(False)

    with PdfPages(fig_dir_path / filename) as pdf:
        plt.tight_layout()
        pdf.savefig(bbox_inches='tight')

    plt.close()

# Create plots for each dataset
for data, filename, ylim in data_sets:
    create_plot(data, filename, ylim)